In [1]:
!pip install tensorflow_gpu==1.15

     |████████████████████████████████| 411.5MB 38kB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 3.8MB 54.2MB/s 
     |████████████████████████████████| 512kB 30.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=ca8a629b04baeb5985efe7c97702100e6c66b4ea5d7d213dec1903650ce4f678
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.3.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorflow-estimator<2.4.0,>=2.3.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing i

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import tempfile
import numpy as np
import math

from tensorflow.examples.tutorials.mnist import input_data
from legendre_batch_norm import legendre

import tensorflow as tf
np.random.seed(0)
tf.set_random_seed(0)


FLAGS = None
def include_activation(name):
    return ('activation_attention' in name)

def lr(iteration):
  if iteration < 5000:
    return 1e-3
  elif iteration < 10000:
      return 1e-3
  elif iteration <  15000:
      return  1e-4  
  if iteration<20000:
        return 1e-5
  return 1e-5


import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# Relu_W_file = np.genfromtxt('Relu_W.csv', delimiter=',')
# Relu_B_file = np.genfromtxt('Relu_B.csv', delimiter=',')

# Relu_W = Relu_W_file#.astype(dtype = 'float64')#[:,0]
# Relu_B = Relu_B_file#.astype(dtype = 'float64')#[:,0]
# print("-----------------")
# print(Relu_W.shape)
# print("-----------------")

def deepnn(x, is_train):
  with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

  # First convolutional layer - maps one grayscale image to 32 feature maps.
  with tf.name_scope('conv1'):
    W_conv1 = weight_variable([5,5, 1, 16])
    b_conv1 = bias_variable([16])
    temp_1 = b_conv1
    #a = tf.reshape(tf.size(x_image), [-1, 1])
    temp_1 = (conv2d(x_image, W_conv1) + b_conv1) #UC
    print('------------')
    #print(temp1)
    # _, _,out,_, _ = non_linear_Ac(temp_1, input_size = 25088, k1 = 4, k2 = 4, output_size = 25088) #UC
    # h_conv1, w1, b1 = RAMO(temp_1, k1=4, k2=4, scale=None) #UC
    h_conv1 = legendre(temp_1, k=3, is_train=is_train, name="Ac/1")
    #h_conv1 = tf.nn.relu(temp_1)

    print(h_conv1, "PVR")
    # h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) #C

  # Pooling layer - downsamples by 2X.
  with tf.name_scope('pool1'):
    h_pool1 = max_pool_2x2(h_conv1)

  # Second convolutional layer -- maps 16 feature maps to 64.
  with tf.name_scope('conv2'):
    W_conv2 = weight_variable([5, 5, 16, 32])
    b_conv2 = bias_variable([32])
    temp_c2 = (conv2d(h_pool1, W_conv2) + b_conv2) #UC
    # h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)  #C
    # _,_,out_c2,_, _ = non_linear_Ac(temp_c2, input_size = 14*14*32, k1 = 4, k2 = 4, output_size = 14*14*32) #UC
    # h_conv2 = tf.reshape(out_c2, [-1, 14, 14, 32]) #UC
    h_conv2 = legendre(temp_c2, k=3, is_train=is_train, name="Ac/2")
    #h_conv2 = tf.nn.relu(temp_c2)

  # Second pooling layer.
  with tf.name_scope('pool2'):
    h_pool2 = max_pool_2x2(h_conv2)

  # Fully connected layer 1 -- after 2 round of downsampling, our 28x28 image
  # is down to 7x7x32 feature maps -- maps this to 1024 features.
  with tf.name_scope('fc1'):
    W_fc1 = weight_variable([7 * 7 * 32, 1024])
    b_fc1 = bias_variable([1024])

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*32]) 
    h_fc1 = tf.nn.elu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1) #C
    #h_fc1 = tf.nn.softmax(h_fc)

  #with tf.name_scope('dropout'):
  #  keep_prob = tf.placeholder(tf.float32)
  #  h_fc1_drop = tf.nn.dropout(h_fc1,0.6)
  #with tf.name_scope('fc3'):
  #  W_fc3 = weight_variable([128,64])
  #  b_fc3 = bias_variable([64])

   # h_pool3_flat = tf.reshape(h_fc1_drop, [-1, 128]) 
   # h_fc3 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc3) + b_fc3) #C
    #h_fc3 = tf.nn.softmax(h_fc)
 # with tf.name_scope('dropout'):
  #  keep_prob = tf.placeholder(tf.float32)
  #  h_fc3_drop = tf.nn.dropout(h_fc3, 0.65)
  # Map the 1024 features to 10 classes, one for each digit
  with tf.name_scope('fc2'):
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])

    y_conv = tf.matmul(h_fc1, W_fc2) + b_fc2
  print('trainable variables')
  total_parameters = 0
  for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    print(variable)
    print(shape)
    variable_parameters = 1
    for dim in shape:
        variable_parameters *= dim.value
    print(variable_parameters)
    total_parameters += variable_parameters
  print(total_parameters)  
  return y_conv


def conv2d(x, W):
  """conv2d returns a 2d convolution layer with full stride."""
  return tf.nn.conv2d(x, W,  strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
  """max_pool_2x2 downsamples a feature map by 2X."""
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def weight_variable(shape):
  """weight_variable generates a weight variable of a given shape."""
  initial = tf.random.truncated_normal(shape, name="scalar_a", dtype=tf.float32, mean=0, stddev=.05)
  #initial = tf.truncated_normal(shape, stddev=0.1)
  #W = tf.get_variable("W", shape=[784, 256],
           #initializer=tf.contrib.layers.xavier_initializer())
  return tf.Variable(initial)

def bias_variable(shape):
  """bias_variable generates a bias variable of a given shape."""
  initial = tf.constant(0.1, shape=shape)
  #initial = tf.random.truncated_normal(shape, name="scalar_a", dtype=tf.float32, mean=0, stddev=.1)
  return tf.Variable(initial)
def main(_):
  mnist = input_data.read_data_sets('data/fashion', source_url='http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/',one_hot=True)
  #mnist = input_data.read_data_sets("/content/fmnist", one_hot=True)
  # mnist = tf.keras.datasets.fashion_mnist.load_data()
  # Create the model
  x = tf.placeholder(tf.float32, [None, 784])

  # Define loss and optimizer
  y_ = tf.placeholder(tf.float32, [None, 10])
  is_train = tf.placeholder(tf.bool)

  learning_rate = tf.placeholder(tf.float32)

  # Build the graph for the deep net
  y_conv = deepnn(x, is_train=is_train)
  keep_prob = tf.placeholder(tf.float32)
  with tf.name_scope('loss'):
    # reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    # l2_loss = .005 * tf.add_n(
    #   # loss is computed using fp32 for numerical stability.
    #   [tf.nn.l2_loss(tf.cast(v, tf.float32)) for v in tf.trainable_variables()
    #    if include_activation(v.name)])
    print([v.name for v in tf.trainable_variables() if include_activation(v.name)])
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_,
                                                            logits=y_conv)
  cross_entropy = tf.reduce_mean(cross_entropy)

  with tf.name_scope('adam'):
    #t1 = tf.compat.v1.train.GradientOptimizer(learning_rate).minimize(cross_entropy)
    t2 = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
    # train_step = tf.group(t1, t2)
    train_step = t2

  with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
  accuracy = tf.reduce_mean(correct_prediction)
  graph_location = tempfile.mkdtemp()
  print('Saving graph to: %s' % graph_location)
  train_writer = tf.summary.FileWriter(graph_location)
  train_writer.add_graph(tf.get_default_graph())
  acc = []
  config = tf.ConfigProto()
  config.gpu_options.allow_growth=True
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(25000):
      batch = mnist.train.next_batch(50)
      if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x: batch[0], y_: batch[1], keep_prob: 0.8, is_train: True})
        print('step %d, training accuracy %g' % (i, train_accuracy))


        a1 = accuracy.eval(feed_dict={
        x: mnist.test.images[0:5000, :], y_: mnist.test.labels[0:5000, :], keep_prob: 1.0, is_train: False})
        a2 = accuracy.eval(feed_dict={
            x: mnist.test.images[5000:, :], y_: mnist.test.labels[5000:, :], keep_prob:1.0, is_train: False})
        print(a1/2 + a2/2)
        acc.append(a1/2 + a2/2)

      # print(batch[0][0])
      train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1, learning_rate: lr(i), is_train: True})

    a1 = accuracy.eval(feed_dict={
        x: mnist.test.images[0:5000, :], y_: mnist.test.labels[0:5000, :], keep_prob: 1.0, is_train: False})
    a2 = accuracy.eval(feed_dict={
        x: mnist.test.images[5000:, :], y_: mnist.test.labels[5000:, :], keep_prob: 1.0, is_train: False})
    print(a1/2 + a2/2)

    np.save('testing_score_taylor_batch_norm', np.array(acc))

if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  parser.add_argument('--data_dir', type=str,
                      default='/content/drive2/My Drive/fashion',
                      help='Directory for storing input data')
  FLAGS, unparsed = parser.parse_known_args()
  tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


W0816 16:59:00.318971 140161899140992 deprecation.py:323] From <ipython-input-2-dfd910e6ae37>:155: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Instructions for updating:
Please write your own downloading logic.


W0816 16:59:00.321214 140161899140992 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.


Instructions for updating:
Please use urllib or similar directly.


...

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


W0816 16:59:06.011965 140161899140992 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/math_grad.py:1375: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Saving graph to: /tmp/tmppoadxcf9
step 0, training accuracy 0.04
0.10000000149011612
step 100, training accuracy 0.84
0.7960000038146973
step 200, training accuracy 0.88
0.7982999980449677
step 300, training accuracy 0.84
0.8217999935150146
step 400, training accuracy 0.94
0.8343999981880188
step 500, training accuracy 0.88
0.8427000045776367
step 600, training accuracy 0.9
0.8557000160217285
step 700, training accuracy 0.94
0.8672000169754028
step 800, training accuracy 0.86
0.8501999974250793
step 900, training accuracy 0.86
0.8535999953746796
step 1000, training accuracy 0.86
0.872299998998642
step 1100, training accuracy 0.92
0.8819000124931335
step 1200, training accuracy 0.84
0.8768999874591827
step 1300, training accuracy 0.94
0.8677000105381012
step 1400, training accuracy 0.82
0.880299985408783
step 1500, training accuracy 0.88
0.8711000084877014
step 1600, training accuracy 0.86
0.8844000101089478
step 1700, training accuracy 0.86
0.8869999945163727
step 1800, training accura

SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
